In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///croot/scikit-learn_1680198742267/work
scikit-learn-intelex==20230426.111118


In [11]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)


[18.52778307 23.06578208 33.68635854 23.75743618 21.49290439 10.87341161
 21.52087965 10.16400086 17.52115575 16.90379088]


In [16]:
print(f'Q1. Answer: standard deviation: {np.std(y_pred):.2f}')

standard deviation: 5.28


In [18]:

from datetime import datetime
today = datetime.today()
df['ride_id'] = f'{today.year:04d}/{today.month:02d}_' + df.index.astype('str')
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

output_file = '04-deployment-Q2-predictions.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)



In [20]:
!ls -hl
# Q2. answer 58M

total 58M
-rw-rw-r-- 1 ubuntu ubuntu  58M Jun 19 09:15 04-deployment-Q2-predictions.parquet
-rw-rw-r-- 1 ubuntu ubuntu   77 Jun 19 08:56 Dockerfile
-rw-rw-r-- 1 ubuntu ubuntu  17K Jun 19 08:56 model.bin
-rw-rw-r-- 1 ubuntu ubuntu 4.5K Jun 19 09:16 starter.ipynb


Q3. Answer: !jupyter nbconvert --to python starter.ipynb

In [21]:
!jupyter nbconvert --to python starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to python
[NbConvertApp] Writing 1707 bytes to starter.py


Q4. Answer: 065e9673e24e0dc5113e2dd2b4ca30c9d8aa2fa90f4c0597241c93b63130d233
Q5. Answer: 12.76
Q6. Answer: 12.83